# Exponential Smoothing

---
1. Single Exponential Smoothing (SES)
      a. Trends the data with a smoothing factor alpha, which is the degree to which data should be followed
      b. 0 < alpha < 1
      c. Y(i+1) = Y(i) + alpha * (Y_pred - Y_actual)
2. Double Exponential Smoothing (Holtz)
      a. Captures Trends in data as well as a term beta to add a degree of dampening
      a. addititve and multiplicative forms are present which capture different types of behaviors depending on the time series you're trying to forecast.

3. Triple Exponential Smoothing (Holtz-Winters)
      a. Captures Trends in data, a degree of dampening, and a factor controls the influence of the seasonal component
  


Source: https://machinelearningmastery.com/exponential-smoothing-for-time-series-forecasting-in-python/





In [ ]:
import numpy      as np
import pandas     as pd
import matplotlib.pyplot as plt
import seaborn    as sns
from   zipfile    import ZipFile
from   datetime   import datetime
import random
import plotly.express as px

In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files = files.upload()

In [ ]:
! mkdir ~/.kaggle          # Making New File Directory
!cp kaggle.json ~/.kaggle/ #New File Path
! chmod 600 ~/.kaggle/kaggle.json  ## Permission

In [ ]:
!kaggle datasets download -d fedesoriano/electric-power-consumption

In [ ]:
with ZipFile('electric-power-consumption.zip') as zip:
  print(zip.namelist())
  with zip.open(zip.namelist()[0]) as csv:
    data = pd.read_csv(csv)
data['Datetime'] = pd.to_datetime(data['Datetime'])
data

In [ ]:
def datetime_simpifier(power_column):

  day_init         = 1
  cumulative_power = 0
  daily_power      = pd.DataFrame()
  date_list        = []
  cumulative_powerlist = []

  for index in data.index:
    day = data.loc[index, 'Datetime'].day
    if day_init == day:
      cumulative_power += data.loc[index, power_column]
    else:

      day          = str(data.loc[index, 'Datetime'].day)
      month        = str(data.loc[index, 'Datetime'].month)
      year         = str(data.loc[index, 'Datetime'].year)
      current_date = datetime.strptime(day + '-' + month + '-' + year, '%d-%m-%Y')
      date_list.append(current_date)
      cumulative_powerlist.append(cumulative_power)

      day_init         = int(day)
      cumulative_power = 0


  daily_power[power_column] = cumulative_powerlist
  daily_power.index         = date_list


  return  daily_power

time_zone_1 = datetime_simpifier('PowerConsumption_Zone1')
time_zone_2 = datetime_simpifier('PowerConsumption_Zone2')
time_zone_3 = datetime_simpifier('PowerConsumption_Zone3')

power_consumption = pd.concat([time_zone_1, time_zone_2, time_zone_3], axis = 1)
power_consumption

**Preprocess and Add Consumption for 24 hour Periods**

In [ ]:
fig = px.line(power_consumption, x = power_consumption.index, y = power_consumption['PowerConsumption_Zone1'])
fig.update_xaxes(rangeslider_visible = True)
fig.show()

In [ ]:
fig = px.line(power_consumption, x = power_consumption.index, y = power_consumption['PowerConsumption_Zone2'])
fig.update_xaxes(rangeslider_visible = True)
fig.show()

In [ ]:
fig = px.line(power_consumption, x = power_consumption.index, y = power_consumption['PowerConsumption_Zone3'])
fig.update_xaxes(rangeslider_visible = True)
fig.show()

**Exponential Smoothing**

In [ ]:
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [ ]:
power_consumption['PowerConsumption_Zone1']

2017-01-02    4.098993e+06
2017-01-03    4.130503e+06
2017-01-04    4.374331e+06
2017-01-05    4.391678e+06
2017-01-06    4.407955e+06
                  ...     
2017-12-26    4.289995e+06
2017-12-27    4.294972e+06
2017-12-28    4.288389e+06
2017-12-29    4.331510e+06
2017-12-30    4.178920e+06
Name: PowerConsumption_Zone1, Length: 363, dtype: float64

In [ ]:
def plot_forecasts(power_zone, option = None, simulation_points = 100):

  # Follows better if data is random

  if option is None:
    fit = SimpleExpSmoothing(power_zone, initialization_method="estimated").fit()

  # Holts should be used when data has a trend, seasonality, and is not random
  elif option.lower() == 'holt':
    fit = Holt(power_zone, damped_trend=True, initialization_method="estimated").fit(
    smoothing_level=0.8, smoothing_trend=0.2)

  fit.forecast(len(power_consumption.index)).rename(r"$\alpha=%s$" % fit.model.params["smoothing_level"])
  simulations = fit.simulate(simulation_points, repetitions=1, anchor = 'end', error = 'additive')

  plt.plot(power_consumption.index, power_zone, label = 'Zone Actual')
  plt.plot(power_consumption.index, fit.fittedvalues, label = 'Fit')
  plt.plot(simulations.index, simulations, label = 'Forecast')
  plt.legend()

  return fit.fittedvalues


plot_forecasts(power_consumption['PowerConsumption_Zone1'], option= None, simulation_points = 400)


In [ ]:
plot_forecasts(power_consumption['PowerConsumption_Zone2'],  option= None, simulation_points = 300)


In [ ]:
plot_forecasts(power_consumption['PowerConsumption_Zone3'],  option= None, simulation_points = 300)

In [ ]:
plot_forecasts(power_consumption['PowerConsumption_Zone3'],  option= None)